In [6]:
#Importing libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [7]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.9.0.post1-cp311-cp311-win_amd64.whl (13.8 MB)



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install numpy==1.25.0

                                              0.0/15.0 MB ? eta -:--:--
                                              0.2/15.0 MB 3.9 MB/s eta 0:00:04
     -                                        0.5/15.0 MB 5.8 MB/s eta 0:00:03
     --                                       0.8/15.0 MB 6.5 MB/s eta 0:00:03
     --                                       1.0/15.0 MB 6.6 MB/s eta 0:00:03
     --                                       1.0/15.0 MB 6.6 MB/s eta 0:00:03
     --                                       1.0/15.0 MB 6.6 MB/s eta 0:00:03
     --                                       1.0/15.0 MB 6.6 MB/s eta 0:00:03
     --                                       1.0/15.0 MB 6.6 MB/s eta 0:00:03
     ---                                      1.5/15.0 MB 3.4 MB/s eta 0:00:04
     -----                                    1.9/15.0 MB 4.3 MB/s eta 0:00:04
     -----                                    2.1/15.0 MB 4.4 MB/s eta 0:00:03
     -----                                    2.1/15.0 MB 4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModel
import faiss
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
embedding_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embedding_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

tokenizer.pad_token = tokenizer.eos_token

def embed_texts(texts, max_length=512):
    inputs = embedding_tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    with torch.no_grad():
        outputs = embedding_model(**inputs)
        if hasattr(outputs, 'pooler_output'):
            embeddings = outputs.pooler_output
        elif hasattr(outputs, 'last_hidden_state'):
            embeddings = outputs.last_hidden_state.mean(dim=1)
        else:
            raise ValueError("The model output does not have 'pooler_output' or 'last_hidden_state'")
    return embeddings.numpy()

dataset = []
with open('dataset.txt', 'r') as file:
    content = file.read()
    dataset = content.split('\n')  

if len(dataset) < 2:
    raise ValueError("The dataset must contain at least two documents.")

print(f"Dataset length: {len(dataset)}")

embeddings = embed_texts(dataset)

print(f"Embeddings shape: {embeddings.shape}")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

def retrieve(query, k=5):
    query_embedding = embed_texts([query])
    distances, indices = index.search(query_embedding, k)
    return [dataset[i] for i in indices[0] if 0 <= i < len(dataset)]

def generate_response(query):
    retrieved_docs = retrieve(query)
    context = " ".join(retrieved_docs)
    input_ids = tokenizer.encode(context + tokenizer.eos_token + query, return_tensors='pt', max_length=1024, truncation=True)
    output = gpt2_model.generate(input_ids, max_length=100, num_return_sequences=1, 
                                 pad_token_id=tokenizer.eos_token_id, 
                                 temperature=0.7, top_p=0.9, top_k=50)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response



Dataset length: 149


Input length of input_ids is 251, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Embeddings shape: (149, 384)
AI in the Business  Artificial Intelligence (AI) Revolution Three key attributes of AI technology. The simulation of human intelligence by machines can infer a fast solution for the problems that are often faced by humanity. AI is backed by advanced data analytics and machine learning, which means AI can learn and gain new insights as it keeps feeding on new data. With proper input, AI could come up with prompt and accurate decisions. In addition to that, the intelligence attribute of AI promotes productivity and reduces dependency on human support which makes AI highly autonomous and a convenient tool to have. Intentionality is often deemed as the technical and ontological attributes of computer programs that derived from the algorithms and knowledge engineering. This attribute can be interpreted as AIâ€™s capability of delivering insights from the real time information and reacting in the way similar to its creatorsâ€™ and usersâ€™. The responses usually 

In [22]:
query = "Intentionality is often deemed as the technical and ontological attributes of computer programs is this true?"
response = generate_response(query)
print(response)

Input length of input_ids is 264, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Intentionality is often deemed as the technical and ontological attributes of computer programs that derived from the algorithms and knowledge engineering. This attribute can be interpreted as AIâ€™s capability of delivering insights from the real time information and reacting in the way similar to its creatorsâ€™ and usersâ€™. The responses usually strongly reflect the social context that creator and users are in. Additionally, with development of data ingestion, storage capacity, processing speed and analytic techniques, AI gets more capable of responding to the issues with increasing sophistication. This very much differentiates AI with the fundamental function of machines that merely carry out predetermined routines. Intentionality Undoubtedly, the artificial intelligence revolutions had profoundly impacted the way businesses operate. The most common practises are the automation of repetitive tasks that require less human input. However, with the consistent improvement of algorithm